# Event Analytics

## Authentication

In [5]:
import keyring
from trendminer_interface import TrendMinerClient

url = 'https://cs.trendminer.net/'
client_id = 'wdanielsclient'

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz = "Europe/Brussels"
)

## Selecting tags
We can load tags directly from their names. 

In [6]:
flow = client.tag.from_name("TM4-HEX-FI0620")
temp = client.tag.from_name("TM4-HEX-TI0620")

## Selecting intervals
This package uses pandas [Interval](https://pandas.pydata.org/docs/reference/api/pandas.Interval.html) and [IntervalIndex](https://pandas.pydata.org/docs/reference/api/pandas.IntervalIndex.html#pandas.IntervalIndex) as central objects for denoting events and as method inputs.

Importing from `trendminer_interface` adds [custom accessors](https://pandas.pydata.org/docs/development/extending.html#registering-custom-accessors) to the DataFrame class, which contain TrendMiner-specific methods as well as some utility methods.

A common starting point is therefore to generate an IntervalIndex with a regular frequency, and use that to create a new DataFrame. Here, we select all full weeks (starting Monday) in April 2023.

In [7]:
import pandas as pd

week_index = pd.interval_range(
    start=pd.Timestamp("2023-04-01 00:00:00", tz=client.tz),
    end=pd.Timestamp("2023-05-01 00:00:00", tz=client.tz),
    freq="W-MON",
    closed="left",
)

df = pd.DataFrame(index=week_index)

## Calculations on intervals
We can now perform calculations usingt the custom `interval` accessor on the DataFrame. Accessor methods will return the manipulated DataFrame.

In [8]:
df.interval.calculate(tag=temp, operation="max", name="max temp")

max temp
[2023-04-03 00:00:00+02:00, 2023-04-10 00:00:00...  52.869370
[2023-04-10 00:00:00+02:00, 2023-04-17 00:00:00...  51.089703
[2023-04-17 00:00:00+02:00, 2023-04-24 00:00:00...  50.764526
[2023-04-24 00:00:00+02:00, 2023-05-01 00:00:00...  50.890480

To add multiple calculations, we can pipe DataFrame operations. We can of course include regular Python operations on the DataFrame. We can integrate the weekly flow to get the total volume, but because the TrendMiner integral assumes a time unit of 'day', and our flow is expressed in m3 per hour, we need to multiply by 24 to get the volume in m3. Then as an example, let's set some bound on the flow derived from the mean and standard deviation, keeping into account that the lower limit should never be lower than 0.

In [29]:
df_bounds = (
    pd.DataFrame(index=week_index)
    .interval.calculate(tag=flow, operation="int", name="total")
    .assign(volume=lambda df: df["total"]*24)
    .interval.calculate(tag=flow, operation="mean", name="mean")
    .interval.calculate(tag=flow, operation="std", name="std")
    .assign(upper_limit=lambda df: df["mean"]+1.5*df["std"])
    .assign(lower_limit=lambda df: (df["mean"]-1.5*df["std"]).clip(lower=0))
    .round(1)
)

df_bounds

total  volume  mean   std  \
[2023-04-03 00:00:00+02:00, 2023-04-10 00:00:00...  171.7  4121.8  24.5  17.5   
[2023-04-10 00:00:00+02:00, 2023-04-17 00:00:00...  338.3  8118.0  48.3  26.9   
[2023-04-17 00:00:00+02:00, 2023-04-24 00:00:00...  360.9  8661.6  51.6  29.0   
[2023-04-24 00:00:00+02:00, 2023-05-01 00:00:00...  300.9  7222.0  43.0  21.7   

                                                    upper_limit  lower_limit  
[2023-04-03 00:00:00+02:00, 2023-04-10 00:00:00...         50.8          0.0  
[2023-04-10 00:00:00+02:00, 2023-04-17 00:00:00...         88.7          8.0  
[2023-04-17 00:00:00+02:00, 2023-04-24 00:00:00...         95.0          8.1  
[2023-04-24 00:00:00+02:00, 2023-05-01 00:00:00...         75.5         10.5

## Searches

## Context items

## Interval manipulations

## Writing to a new tag
We can create tags using the csv import endpoint. First we need to instantiate a new tag object.

In [23]:
tag = client.tag(
    name=f"Example lower limit",
    description="average batch temperature",
    tag_type="DISCRETE",
)

We also need a pandas Series with the tag data. This is easily generated from the DataFrame of calculations.

In [30]:
ser = pd.Series(index=df_bounds.index.left, data=df_bounds["lower_limit"], name=tag.name)
ser.head()

2023-04-03 00:00:00+02:00     0.0
2023-04-10 00:00:00+02:00     8.0
2023-04-17 00:00:00+02:00     8.1
2023-04-24 00:00:00+02:00    10.5
Freq: W-MON, Name: Example lower limit, dtype: float64

Then we can save/update this csv tag on the server

In [28]:
tag_data_dict = {tag: ser}
client.io.tag.save(tag_data_dict)